# Interferometry Basics for OCT

In this notebook, we'll explore the physics of interference and coherence in greater depth, providing the foundation for understanding how OCT works.

## Learning Objectives

By the end of this notebook, you will understand:
1. Wave interference and superposition
2. Temporal and spatial coherence
3. The difference between high and low coherence sources
4. How interference patterns reveal depth information

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from scipy.signal import hilbert

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Libraries loaded successfully!")

## 1. Wave Superposition and Interference

When two waves meet, they combine according to the **superposition principle**:

$$E_{total} = E_1 + E_2$$

For two coherent waves:
$$E_1 = A_1 \cos(\omega t + \phi_1)$$
$$E_2 = A_2 \cos(\omega t + \phi_2)$$

The intensity (what we detect) is:
$$I = |E_{total}|^2 = I_1 + I_2 + 2\sqrt{I_1 I_2}\cos(\phi_2 - \phi_1)$$

The last term is the **interference term** - it can be positive (constructive) or negative (destructive).

In [ ]:
# Demonstrate wave interference
def wave_interference(amplitude1, amplitude2, phase_diff):
    """
    Simulate interference between two waves.
    """
    t = np.linspace(0, 4*np.pi, 1000)
    
    # Two waves
    wave1 = amplitude1 * np.cos(t)
    wave2 = amplitude2 * np.cos(t + phase_diff)
    
    # Combined wave
    combined = wave1 + wave2
    
    return t, wave1, wave2, combined

# Create figure with different phase differences
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
phase_diffs = [0, np.pi/2, np.pi, 3*np.pi/2]
titles = ['Constructive (Δφ = 0)', 'Partial (Δφ = π/2)', 
          'Destructive (Δφ = π)', 'Partial (Δφ = 3π/2)']

for ax, phase_diff, title in zip(axes.flat, phase_diffs, titles):
    t, w1, w2, combined = wave_interference(1, 1, phase_diff)
    
    ax.plot(t, w1, 'b--', alpha=0.5, label='Wave 1', linewidth=1.5)
    ax.plot(t, w2, 'r--', alpha=0.5, label='Wave 2', linewidth=1.5)
    ax.plot(t, combined, 'g-', label='Combined', linewidth=2)
    ax.set_xlabel('Time (arbitrary units)')
    ax.set_ylabel('Amplitude')
    ax.set_title(title, fontweight='bold')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-2.5, 2.5)

plt.tight_layout()
plt.show()

print("Notice how the combined wave amplitude depends on the phase difference!")
print("This is the basis of interferometry.")

## 2. Temporal Coherence

**Temporal coherence** describes how well a wave maintains its phase over time.

### Coherence Time ($\tau_c$)
The time over which the wave maintains phase coherence:
$$\tau_c = \frac{1}{\Delta \nu}$$

where $\Delta \nu$ is the frequency bandwidth.

### Coherence Length ($l_c$)
The distance light travels during the coherence time:
$$l_c = c \cdot \tau_c = \frac{c}{\Delta \nu} = \frac{\lambda_0^2}{\Delta \lambda}$$

For OCT, we want **low temporal coherence** (short $l_c$) for good depth resolution!

In [ ]:
# Compare high vs. low coherence sources
def generate_light_source(wavelength, bandwidth, num_points=10000):
    """
    Generate a light source spectrum.
    
    Parameters:
    - wavelength: Center wavelength (nm)
    - bandwidth: Spectral width (nm)
    - num_points: Number of spectral points
    """
    # Wavelength array
    lambda_range = np.linspace(wavelength - 2*bandwidth, 
                                wavelength + 2*bandwidth, 
                                num_points)
    
    # Gaussian spectrum
    spectrum = np.exp(-((lambda_range - wavelength) / bandwidth)**2)
    
    return lambda_range, spectrum

# Create comparison plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# High coherence (laser)
lambda1, spec1 = generate_light_source(840, 0.1)  # Very narrow bandwidth
axes[0].plot(lambda1, spec1, 'r', linewidth=2)
axes[0].set_xlabel('Wavelength (nm)')
axes[0].set_ylabel('Intensity (normalized)')
axes[0].set_title('High Coherence Source (Laser)\nΔλ = 0.1 nm', fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim(835, 845)

# Calculate coherence length
lc_high = (840**2) / 0.1 / 1000  # in micrometers
axes[0].text(0.05, 0.95, f'lc ≈ {lc_high:.0f} μm\n(Very long!)', 
             transform=axes[0].transAxes, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Low coherence (LED/SLD)
lambda2, spec2 = generate_light_source(840, 50)  # Broad bandwidth
axes[1].plot(lambda2, spec2, 'b', linewidth=2)
axes[1].set_xlabel('Wavelength (nm)')
axes[1].set_ylabel('Intensity (normalized)')
axes[1].set_title('Low Coherence Source (SLD)\nΔλ = 50 nm', fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim(700, 980)

# Calculate coherence length
lc_low = (840**2) / 50 / 1000  # in micrometers
axes[1].text(0.05, 0.95, f'lc ≈ {lc_low:.1f} μm\n(Perfect for OCT!)', 
             transform=axes[1].transAxes, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"High coherence: lc = {lc_high:.0f} μm (too long for OCT)")
print(f"Low coherence: lc = {lc_low:.1f} μm (ideal for OCT depth resolution)")

## 3. Interference with Low-Coherence Light

With low-coherence light, interference only occurs when:
- Path lengths are matched within the coherence length
- This creates a **localized interference region**

This is the key to OCT's depth selectivity!

The interference signal can be written as:
$$I(\Delta z) = I_0 \left[1 + \gamma(\Delta z) \cos\left(\frac{4\pi}{\lambda_0}\Delta z\right)\right]$$

where:
- $\gamma(\Delta z)$ is the coherence envelope (Gaussian for typical sources)
- $\Delta z$ is the path length difference

In [ ]:
# Simulate interferogram with different coherence lengths
def interferogram(delta_z, wavelength, coherence_length):
    """
    Calculate the interference signal.
    
    Parameters:
    - delta_z: Path length difference array (meters)
    - wavelength: Center wavelength (meters)
    - coherence_length: Coherence length (meters)
    """
    # Coherence envelope (Gaussian)
    envelope = np.exp(-(delta_z / coherence_length)**2)
    
    # Carrier fringes
    carrier = np.cos(4 * np.pi * delta_z / wavelength)
    
    # Combined signal
    I = 1 + envelope * carrier
    
    return I, envelope, carrier

# Simulation parameters
wavelength = 840e-9  # 840 nm
delta_z = np.linspace(-50e-6, 50e-6, 5000)  # ±50 μm range

# Different coherence lengths
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
coherence_lengths = [20e-6, 10e-6, 5e-6, 2e-6]  # Different lc values

for ax, lc in zip(axes.flat, coherence_lengths):
    I, envelope, carrier = interferogram(delta_z, wavelength, lc)
    
    ax.plot(delta_z * 1e6, I, 'b', linewidth=1.5, label='Interferogram')
    ax.plot(delta_z * 1e6, 1 + envelope, 'r--', linewidth=2, label='Envelope')
    ax.plot(delta_z * 1e6, 1 - envelope, 'r--', linewidth=2)
    ax.set_xlabel('Path Length Difference (μm)')
    ax.set_ylabel('Intensity (a.u.)')
    ax.set_title(f'Coherence Length = {lc*1e6:.1f} μm', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-0.1, 2.1)

plt.tight_layout()
plt.show()

print("Key observations:")
print("1. Shorter coherence length → Narrower interference region")
print("2. This narrow region provides depth selectivity")
print("3. The envelope width determines axial resolution")

## 4. Multiple Reflectors: Simulating Tissue Structure

Real tissue has multiple reflecting layers. Each layer creates its own interference pattern when the reference mirror matches its depth.

The total signal is the sum of contributions from all layers:
$$I_{total} = \sum_i R_i \cdot \gamma(z - z_i) \cos\left(\frac{4\pi}{\lambda_0}(z - z_i)\right)$$

where $R_i$ is the reflectivity of layer $i$ at depth $z_i$.

In [ ]:
# Simulate multiple reflecting layers
def multi_layer_interferogram(z_ref, layer_depths, layer_reflectivities, 
                               wavelength, coherence_length):
    """
    Simulate interference from multiple tissue layers.
    
    Parameters:
    - z_ref: Reference arm positions (array)
    - layer_depths: List of layer depths
    - layer_reflectivities: List of reflectivity values
    - wavelength: Center wavelength
    - coherence_length: Coherence length
    """
    signal = np.ones_like(z_ref)  # DC component
    
    for depth, reflectivity in zip(layer_depths, layer_reflectivities):
        delta_z = z_ref - depth
        envelope = np.exp(-(delta_z / coherence_length)**2)
        carrier = np.cos(4 * np.pi * delta_z / wavelength)
        signal += reflectivity * envelope * carrier
    
    return signal

# Define a realistic tissue structure
layer_depths = [50e-6, 100e-6, 180e-6, 250e-6]  # μm
layer_names = ['Surface', 'Layer 1', 'Layer 2', 'Layer 3']
layer_reflectivities = [0.6, 0.4, 0.5, 0.3]

# Reference arm scan range
z_ref = np.linspace(0, 300e-6, 3000)

# Generate signal
wavelength = 840e-9
coherence_length = 7e-6
signal = multi_layer_interferogram(z_ref, layer_depths, layer_reflectivities,
                                   wavelength, coherence_length)

# Plot
plt.figure(figsize=(14, 6))
plt.plot(z_ref * 1e6, signal, 'b', linewidth=1)
plt.xlabel('Reference Arm Position (μm)', fontsize=12)
plt.ylabel('Detector Signal (a.u.)', fontsize=12)
plt.title('OCT Interferogram from Multi-Layer Tissue', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Mark layer positions
for depth, name, refl in zip(layer_depths, layer_names, layer_reflectivities):
    plt.axvline(depth * 1e6, color='r', linestyle='--', alpha=0.6)
    plt.text(depth * 1e6, plt.ylim()[1] * 0.95, f'{name}\nR={refl}', 
             ha='center', va='top',
             bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

plt.tight_layout()
plt.show()

print("This is a simulated A-scan!")
print("Each 'burst' of oscillations corresponds to a tissue layer.")
print("The envelope of each burst indicates the layer's reflectivity.")

## 5. Extracting the A-Scan: Demodulation

To recover the tissue structure from the interference signal, we need to extract the **envelope**.

This is typically done using:
1. **Hilbert transform**: Creates an analytic signal
2. **Magnitude**: Extracts the envelope

$$A(z) = |\text{Hilbert}(I(z))|$$

The resulting A-scan shows reflectivity vs. depth!

In [ ]:
# Demodulate the interferogram to get the A-scan
from scipy.signal import hilbert

# Remove DC component
signal_ac = signal - np.mean(signal)

# Apply Hilbert transform
analytic_signal = hilbert(signal_ac)
envelope = np.abs(analytic_signal)

# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Raw interferogram
axes[0].plot(z_ref * 1e6, signal, 'b', linewidth=1, alpha=0.7)
axes[0].set_xlabel('Depth (μm)', fontsize=12)
axes[0].set_ylabel('Interference Signal', fontsize=12)
axes[0].set_title('Raw Interferogram (What the Detector Sees)', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)
for depth in layer_depths:
    axes[0].axvline(depth * 1e6, color='r', linestyle='--', alpha=0.5)

# Demodulated A-scan
axes[1].plot(z_ref * 1e6, envelope, 'g', linewidth=2)
axes[1].set_xlabel('Depth (μm)', fontsize=12)
axes[1].set_ylabel('Reflectivity (a.u.)', fontsize=12)
axes[1].set_title('Demodulated A-Scan (Tissue Structure)', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Mark layer positions and add labels
for depth, name in zip(layer_depths, layer_names):
    axes[1].axvline(depth * 1e6, color='r', linestyle='--', alpha=0.5)
    axes[1].text(depth * 1e6, plt.ylim()[1] * 0.9, name, 
                 ha='center', rotation=0,
                 bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))

plt.tight_layout()
plt.show()

print("✓ The A-scan reveals the tissue layer structure!")
print("✓ Each peak corresponds to a reflecting interface")
print("✓ Peak height indicates reflectivity")
print("✓ Peak position indicates depth")

## 6. Effect of Coherence Length on Resolution

Let's see how coherence length affects our ability to resolve closely spaced layers.

In [ ]:
# Compare resolution with different coherence lengths
# Two closely spaced layers
close_layers = [100e-6, 110e-6]  # 10 μm apart
close_reflectivities = [0.5, 0.5]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
coherence_lengths_test = [15e-6, 10e-6, 5e-6, 3e-6]
titles = ['Poor Resolution (lc=15 μm)', 'Marginal (lc=10 μm)', 
          'Good (lc=5 μm)', 'Excellent (lc=3 μm)']

z_ref_test = np.linspace(80e-6, 130e-6, 2000)

for ax, lc, title in zip(axes.flat, coherence_lengths_test, titles):
    signal_test = multi_layer_interferogram(z_ref_test, close_layers, 
                                            close_reflectivities, wavelength, lc)
    signal_ac = signal_test - np.mean(signal_test)
    envelope_test = np.abs(hilbert(signal_ac))
    
    ax.plot(z_ref_test * 1e6, envelope_test, 'b', linewidth=2)
    ax.axvline(close_layers[0] * 1e6, color='r', linestyle='--', alpha=0.6, label='Layer 1')
    ax.axvline(close_layers[1] * 1e6, color='g', linestyle='--', alpha=0.6, label='Layer 2')
    ax.set_xlabel('Depth (μm)')
    ax.set_ylabel('Reflectivity (a.u.)')
    ax.set_title(title, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Resolution criterion: Two layers can be distinguished when lc/2 < layer spacing")
print(f"Layer spacing: {(close_layers[1] - close_layers[0])*1e6:.1f} μm")
print(f"\nFor lc = 15 μm: Resolution = 7.5 μm (cannot resolve 10 μm spacing)")
print(f"For lc = 3 μm: Resolution = 1.5 μm (clearly resolves 10 μm spacing)")

## Summary

In this notebook, we've learned:

1. ✅ **Wave interference**: How waves combine constructively or destructively

2. ✅ **Temporal coherence**: The key property that enables depth selectivity

3. ✅ **Low vs. high coherence**: Why OCT needs broadband light sources

4. ✅ **Interferograms**: The raw signals from OCT systems

5. ✅ **A-scan extraction**: How to recover depth information via demodulation

6. ✅ **Resolution limits**: How coherence length determines what we can see

## Key Takeaways

- Shorter coherence length (broader bandwidth) → Better axial resolution
- The interference envelope reveals tissue structure
- Signal processing (Hilbert transform) converts interferograms to A-scans
- Resolution is fundamentally limited by coherence length

## Next Steps

In the next notebook, we'll explore:
- Detailed calculations of axial and lateral resolution
- Trade-offs in system design
- How to optimize image quality

---

**Continue to Notebook 3: Resolution and Image Quality →**